In [54]:
import requests
import pandas as pd
import json
pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows', None)
#test change

In [55]:
#get=GEO_ID,NAME,EMP,NATION,PAYANN,STATE,YEAR
apikey = 'af4a7c461bcda111515e0ebeea5e4124fcc267d6'
get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F'
url = f"https://api.census.gov/data/2018/abscs?get={get}&for=us:*&key={apikey}"
r = requests.get(url)
print(r)

r = r.json()

<Response [200]>


In [56]:
#p = pd.read_json(r)

r = json.dumps(r)
p = pd.read_json(r)
p.head(10)

headers = p.iloc[0]
new_p  = pd.DataFrame(p.values[1:], columns=headers)

In [57]:
new_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19458 entries, 0 to 19457
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   GEO_ID            19458 non-null  object
 1   NAME              19458 non-null  object
 2   NAICS2017         19458 non-null  object
 3   NAICS2017_LABEL   19458 non-null  object
 4   SEX               19458 non-null  object
 5   SEX_LABEL         19458 non-null  object
 6   ETH_GROUP         19458 non-null  object
 7   ETH_GROUP_LABEL   19458 non-null  object
 8   RACE_GROUP        19458 non-null  object
 9   RACE_GROUP_LABEL  19458 non-null  object
 10  VET_GROUP         19458 non-null  object
 11  VET_GROUP_LABEL   19458 non-null  object
 12  EMPSZFI           19458 non-null  object
 13  EMPSZFI_LABEL     19458 non-null  object
 14  YEAR              19458 non-null  object
 15  FIRMPDEMP         19458 non-null  object
 16  FIRMPDEMP_F       7516 non-null   object
 17  RCPPDEMP    

In [64]:
new_p.head()

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,...,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F,us
0,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,...,None,3.0,None,8.9,None,7.7,None,7.0,None,1
1,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,...,None,4.7,None,19.2,None,0.0,None,13.5,None,1
2,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,...,None,4.7,None,8.2,None,4.8,None,7.4,None,1
3,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,...,None,5.9,None,4.3,None,5.6,None,6.3,None,1
4,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,...,None,6.0,None,16.2,None,6.7,None,10.7,None,1


,0,1,2,3
0,GEO_ID,NAME,RACE_GROUP,state
1,0400000US28,Mississippi,92,28
2,0400000US28,Mississippi,50,28
3,0400000US28,Mississippi,60,28
4,0400000US42,Pennsylvania,50,42
5,0400000US28,Mississippi,30,28
6,0400000US28,Mississippi,70,28
7,0400000US28,Mississippi,90,28
8,0400000US28,Mississippi,00,28
9,0400000US28,Mississippi,40,28
